Both quantum computing and quantum machine learning, see a qauntum circuit as a function (already seen in the previous notebook), thus we can use the same tools we usually use with the functions, for example first and second order gradient.

In [1]:
import pennylane as qml
from pennylane import numpy as np


In [5]:
dev1  = qml.device("default.qubit", wires=4)
@qml.qnode(dev1)

def function(theta1, theta2, theta3, theta4):
    qml.RX(theta1, wires=0)
    qml.RX(theta2, wires=1)
    qml.RX(theta3, wires=2)
    qml.RX(theta4, wires=3)
    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[1,2])
    qml.CNOT(wires=[2,3])
    qml.CNOT(wires=[3,0])
    qml.RX(theta1, wires=0)
    qml.RX(theta2, wires=1)
    qml.RX(theta3, wires=2)
    qml.RX(theta4, wires=3)
    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[1,2])
    qml.CNOT(wires=[2,3])
    qml.CNOT(wires=[3,0])
    qml.expval(qml.PauliZ(0))

    return qml.state()


print(qml.draw(function)(np.pi/4, np.pi/4, np.pi/4, np.pi/4))


0: ──RX(0.79)─╭●───────╭X──RX(0.79)─╭●───────╭X─┤  <Z>  State
1: ──RX(0.79)─╰X─╭●────│───RX(0.79)─╰X─╭●────│──┤       State
2: ──RX(0.79)────╰X─╭●─│───RX(0.79)────╰X─╭●─│──┤       State
3: ──RX(0.79)───────╰X─╰●──RX(0.79)───────╰X─╰●─┤       State


that circuit done above, can be implemented in a easier way:

In [11]:
n_wires = 4
dev2 = qml.device("default.qubit", wires = n_wires)

@qml.qnode(dev2)
def entangler_circuit(weights):
  qml.BasicEntanglerLayers(weights, wires = range(n_wires))
  return qml.expval(qml.PauliZ(0))


print(qml.draw(entangler_circuit)([[0.1,0.2,0.3,0.4],[0.5,0.6,0.7, 0.8]])) #<--- 8 different parameters for 8 different RX gates
print(qml.draw(entangler_circuit, level = "device")([[0.1,0.2,0.3,0.4],[0.5,0.6,0.7, 0.8]]))

0: ─╭BasicEntanglerLayers(M0)─┤  <Z>
1: ─├BasicEntanglerLayers(M0)─┤     
2: ─├BasicEntanglerLayers(M0)─┤     
3: ─╰BasicEntanglerLayers(M0)─┤     

M0 = 
[[0.1 0.2 0.3 0.4]
 [0.5 0.6 0.7 0.8]]
0: ──RX(0.10)─╭●───────╭X──RX(0.50)─╭●───────╭X─┤  <Z>
1: ──RX(0.20)─╰X─╭●────│───RX(0.60)─╰X─╭●────│──┤     
2: ──RX(0.30)────╰X─╭●─│───RX(0.70)────╰X─╭●─│──┤     
3: ──RX(0.40)───────╰X─╰●──RX(0.80)───────╰X─╰●─┤     


First EXE: 


In [ ]:
dev3 = qml.device("default.qubit", wires = 3)

@qml.qnode(dev3)
def circuit_as_function(params):
    """
    Implements the circuit shown in the codercise statement.
    Args:
    - params (np.ndarray): [theta_0, theta_1, theta_2, theta_3]
    Returns:
    - (np.tensor): <Z0>
    """
    qml.RX(params[0], wires = 0)
    qml.CNOT(wires = [0,1])
    qml.CNOT(wires = [1,2])
    qml.CNOT(wires = [2,0])
    qml.RY(params[1], wires = 0)
    qml.RY(params[2] , wires = 1)
    qml.RY(params[3], wires = 2)

    return qml.expval(qml.PauliZ(0))

print(qml.draw(circuit_as_function, level = "device")(np.array([0.1, 0.2, 0.3, 0.4])))

# angles = np.linspace(0, 4 * np.pi, 200)
# output_values = np.array([circuit_as_function([0.5, t, 0.5, 0.5]) for t in angles]) <--- For the plot

0: ──RX(0.10)─╭●────╭X──RY(0.20)─┤  <Z>
1: ───────────╰X─╭●─│───RY(0.30)─┤     
2: ──────────────╰X─╰●──RY(0.40)─┤     


EXE 2: STRONGLY ENTANGLED STATES

In [22]:
dev7 = qml.device("default.qubit", wires = 4)

@qml.qnode(dev7)
def strong_entangler(weights):
    """
    Applies Strongly Entangling Layers to the default initial state
    Args:
    - weights (np.ndarray): The weights argument for qml.StronglyEntanglingLayers
    Returns:
    - (np.tensor): <Z0>
    """

    qml.StronglyEntanglingLayers(weights, wires=range(4))
    
    return qml.expval(qml.PauliZ(0))
    
shape = qml.StronglyEntanglingLayers.shape(n_layers=2, n_wires=4)
test_weights = np.random.random(size=shape)

print("The output of your circuit with these weights is <0|Z|0>: ", strong_entangler(test_weights))


The output of your circuit with these weights is <0|Z|0>:  0.3916116303420891


Now we seen how a circuit could be used as a function, we can use PennyLane to compute the gradient of the circuit.

There are many ways to find the gradient, and one of the most famous is the parameter-shift rule:

- The simplest parameter shift rule states that, when F represents the expectation value of a quantum circuit with only single-parameter gates then: (dF)/(dtheta) = (F(theta + pi/2) - F(theta - pi/2))/2

there exist also generalizations for multi-parameter gates.

PENNYLANE FINDS GRADIENTS WITH: qml.jacobian




In [15]:
dev4 = qml.device("default.qubit", wires=4)
@qml.qnode(dev4)

#We want only one basic entangled layer, so the arguments of the entangler is a list of one list of only one list of 4 elements
def entangler(weights):
    qml.BasicEntanglerLayers(weights, wires = range(n_wires))
    return qml.expval(qml.PauliZ(0))


test_weights = np.array([[0.1,0.2,0.3,0.4]], requires_grad=True) #<-- i'm telling to pennylane that this argument is differentiable.
print(qml.jacobian(entangler)(test_weights))


[[ 6.93889390e-18 -1.74813749e-01 -2.66766415e-01 -3.64609810e-01]]


In order to use the standard parameter-shift rules we have to change the decorator to the following:
@qml.qnode(dev, interface="autograd", diff_method="parameter-shift")


In [16]:
dev5 = qml.device("default.qubit", wires=4)
@qml.qnode(dev5, interface="autograd", diff_method="parameter-shift")

#We want only one basic entangled layer, so the arguments of the entangler is a list of one list of only one list of 4 elements
def entangler(weights):
    qml.BasicEntanglerLayers(weights, wires = range(n_wires))
    return qml.expval(qml.PauliZ(0))


test_weights = np.array([[0.1,0.2,0.3,0.4]], requires_grad=True) #<-- i'm telling to pennylane that this argument is differentiable.
print(qml.jacobian(entangler)(test_weights))

[[ 0.         -0.17481375 -0.26676641 -0.36460981]]


Sometimes, we may encounter applications in which we want some circuit parameters to be differentiated, while others should remain fixed
For example, let's consider two Basic Entangler layers and differentiate only with respect to the parameters diff_weights in the first layer. 

In [20]:
n_wires = 4
dev6 = qml.device("default.qubit", wires=n_wires)
@qml.qnode(dev6, interface="autograd", diff_method="parameter-shift")

def entangler_fixed(diff_w, fixed_w):
    qml.BasicEntanglerLayers(diff_w, wires=range(n_wires))
    qml.BasicEntanglerLayers(fixed_w, wires=range(n_wires))
    return qml.expval(qml.PauliZ(0))

test_diff_weights = np.array([[0.5,0.1,-0.4,0.6]], requires_grad = True)
test_fixed_weights = np.array([[0.1,0.2,0.3,0.4]], requires_grad = False)

print(qml.jacobian(entangler_fixed)(test_diff_weights, test_fixed_weights))


[[-0.30647646  0.06160872  0.         -0.41303853]]


Exe3: Complete the embedding_and_circuit QNode which depends on a non-trainable array of parameters features and trainable parameters params. The features are the arguments of a qml.AngleEmbedding routine, applied at the start of the circuit to encode some features. Then it is followed by a quantum model that depends on params. PHOTO

In [23]:
dev8 = qml.device("default.qubit", wires = 3)

@qml.qnode(dev8)
def embedding_and_circuit(features, params):
    """
    A QNode that depends on trainable and non-trainable parameters
    Args:
    - features (np.ndarray): Non-trainable parameters in the AngleEmbedding routine
    - params (np.ndarray): Trainable parameters for the rest of the circuit
    Returns:
    - (np.tensor): <Z0>
    """

    qml.AngleEmbedding(features, wires = range(3))
    qml.CNOT(wires = [0,1])
    qml.CNOT(wires = [1,2])
    qml.CNOT(wires = [2,0])
    qml.RY(params[0], wires = 0)
    qml.RY(params[1], wires = 1)
    qml.RY(params[2], wires = 2)
    
    
    return qml.expval(qml.PauliZ(0))

features = np.array([0.3,0.4,0.6], requires_grad = False)
params = np.array([0.4,0.7,0.9], requires_grad = True)
print("The gradient of the circuit is:", qml.jacobian(embedding_and_circuit)(features, params))

The gradient of the circuit is: [-2.96029765e-01  2.77555756e-17  0.00000000e+00]


So far, we have assumed that the measurement of interest is the expectation value of an observable. However, the output of a circuit could be described by more than one component.
For example, if the output of a circuit with K wires are the measurement probabilities, the output can be a real-valued vector o (F_0 ... F_(m-1)) with m = 2^K components.
In this case, the gradient of the output is a matrix with m rows and n columns.
We can use qml.jacobian to make this computation.

In [24]:
dev8 = qml.device("default.qubit", wires = 2)

@qml.qnode(dev8)
def vector_valued_circuit(params):
    qml.RX(params[0], wires = 0)
    qml.CNOT(wires=[0,1])
    qml.RY(params[1], wires = 0)
    return qml.probs(wires = [0,1])

sample_params = np.array([0.1,0.2], requires_grad = True)
print(qml.jacobian(vector_valued_circuit)(sample_params))

[[-0.0494192  -0.09908654]
 [ 0.00049751  0.00024813]
 [-0.00049751  0.09908654]
 [ 0.0494192  -0.00024813]]


In [26]:
dev9 = qml.device("default.qubit", wires = 2)

@qml.qnode(dev9, diff_method = "parameter-shift", max_diff = 2)
def scalar_valued_circuit(params):
  qml.RX(params[0], wires = 0)
  qml.CNOT(wires=[0,1])
  qml.RY(params[1], wires = 0)
  return qml.expval(qml.PauliZ(0))

test_params = np.array([0.7,0.3], requires_grad = True)
qml.jacobian(qml.jacobian(scalar_valued_circuit))(test_params)

array([[-0.73068165,  0.19037934],
       [ 0.19037934, -0.73068165]])

Now that we know how to interpret circuits as functions and take their derivatives, let's use this knowledge to solve optimization problems.
We would like to know what is the minimum expectation value that the circuit output can have.
This means that we're treating the circuit as a cost function, that is, a function we'd like to minimize. 

We will use a gradient discent algorithm.


In [29]:
#Let's begin with a simple differentiable circuit.
dev10 = qml.device("default.qubit", wires = 2)

@qml.qnode(dev10, diff_method = "parameter-shift")
def scalar_valued_circuit(params):
    qml.RX(params[0], wires = 0)
    qml.CNOT(wires=[0,1])
    qml.RY(params[1], wires = 0)
    return qml.expval(qml.PauliZ(0))

def optimize(cost_function, init_params, *steps):

    opt = qml.GradientDescentOptimizer(stepsize = 0.4)
    steps = 100
    params = init_params

    for i in range(steps):
      params = opt.step(cost_function, params)

    return params, cost_function(params)

#This routine returns the parameters for which the cost_function is optimized and the minimum value of cost_function.
initial_parameters = np.array([0.7,0.3], requires_grad = True)
print(optimize(scalar_valued_circuit, initial_parameters, 100))


(tensor([3.14159265e+00, 4.86221075e-17], requires_grad=True), -1.0)
